# MNIST Experiments

This notebook is created to experiment with [MNIST](https://en.wikipedia.org/wiki/MNIST_database) dataset.

Plan:
- Dense network solution on CPU
- Dense network solution on GPU
- CNN+dense network solution on CPU
- CNN+dense network solution on GPU

Tools: Keras, PyTorch

In [2]:
import keras

Using TensorFlow backend.
/Users/alexchernov/anaconda3/envs/deep-learning/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/alexchernov/anaconda3/envs/deep-learning/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/alexchernov/anaconda3/envs/deep-learning/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint1

In [15]:
from keras.datasets import mnist
from keras.layers import Dense, Dropout, Flatten
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [44]:
#get train & tesr dataset
(train_data, train_labels), (test_data, test_labels) = mnist.load_data()

In [45]:
img_width = 28
img_height = 28
img_size = img_width * img_height # 784
train_img_number = train_data.shape[0]
test_img_number = test_data.shape[0]
classes_number = 10 # 10 digits 0-9
batch_size = 128
train_epochs = 10

In [46]:
# shape images
if K.image_data_format() == 'channels_first':
    train_data = train_data.reshape(train_img_number, 1, img_width, img_height)
    test_data = test_data.reshape(test_img_number, 1, img_width, img_height)
    input_shape = (1, img_width, img_height)
else: # channels_last
    train_data = train_data.reshape(train_img_number, img_width, img_height, 1)
    test_data = test_data.reshape(test_img_number, img_width, img_height, 1)
    input_shape = (img_width, img_height, 1)

In [47]:
train_data = train_data.astype('float32')
test_data = test_data.astype('float32')
train_data /= 255
test_data /= 255

In [48]:
print('train shape', train_data.shape)

train shape (60000, 28, 28, 1)


In [49]:
train_labels = keras.utils.to_categorical(train_labels, classes_number)
test_labels = keras.utils.to_categorical(test_labels, classes_number)

In [86]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=input_shape)) # 32x28x28
model.add(Conv2D(64, kernel_size=(3,3), activation='relu')) # 64x28x28
model.add(MaxPooling2D(pool_size=(2,2))) # 64x14x14
model.add(Dropout(0.50))
model.add(Flatten()) # 50176
model.add(Dense(128, activation='relu')) # 128
model.add(Dropout(0.70))
model.add(Dense(classes_number, activation='softmax')) # 10

In [87]:
model.compile(
    loss=keras.losses.categorical_crossentropy,
    optimizer=keras.optimizers.adadelta(), #keras.optimizers.adadelta(),
    metrics=['accuracy']
)

In [68]:
print(test_data.shape)
print(test_labels.shape)

(10000, 28, 28, 1)
(10000, 10)


In [ ]:
train_epochs = 2

model.fit(
    train_data, train_labels, 
    batch_size=batch_size, 
    epochs=train_epochs, 
    verbose=1, 
    validation_data=(test_data, test_labels)
)

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
47616/60000 [======================>.......] - ETA: 29s - loss: 0.4159 - accuracy: 0.8707

In [85]:
score = model.evaluate(test_data, test_labels, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.03707205359062646
Test accuracy: 0.9879999756813049
